<a href="https://colab.research.google.com/github/MELAI-1/Radioastronomy/blob/main/GaussianProcess_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import matplotlib
%matplotlib inline

import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Sampling from a Gaussian process

Let us assume that some field $x(t)$ follows a Gaussian process i.e.

$$ x(t) \sim \mathcal{GP}(m(t), k(t, t')).  $$

How can we draw samples of $x$ at arbitrary points in the domain $t \in \mathbb{R}$?

In [ ]:
# choose a random domain
n = 1000
t = np.linspace(0, 10, n)

# define mean and covariance functions
def mean(t):
    return np.zeros(t.shape)  # zero mean

# squared exponential covariance function
def sqexp(t, tp, sigmaf, l):
    return sigmaf**2 * np.exp(-0.5 * (t - tp)**2 / l**2)


# now we can construct the covariance matrix
sigmaf = 1.0
l = 0.1
K = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        K[i, j] = sqexp(t[i], t[j], sigmaf, l)

# to draw samples we compute a Cholesky decomposition of the covariance matrix
L = np.linalg.cholesky(K + 1e-6*np.eye(n))  # note the jitter term for numerical stability

# draw samples
n_samples = 5
f = mean(t)[:, None] + np.dot(L, np.random.randn(n, n_samples))

# plot samples
plt.figure()
plt.plot(t, f)
plt.xlabel('t')
plt.ylabel('f(t)')
plt.title('Samples from GP')
plt.show()

# 1) - try altering the hyper-parameter values and see how the samples change
# 2) - try altering the mean function and see how the samples change
# 3) - try altering the covariance function and see how the samples change
# 4) - try altering the number of samples and see how the samples change
# 5) - try altering the domain and see how the samples change

# Gaussian process denoising

Given observations (dropping $t$ for simplicity)

$$ y = x + \epsilon, ~~ \epsilon \sim \mathcal{N}(0, \Sigma)  $$

we know that the likelihood for the problem takes the form

$$ p(y | x) = \frac{1}{\sqrt{(2\pi)^n |\Sigma|}} \exp\left(-\frac{1}{2}(y - x)^\dagger \Sigma^{-1} (y - x) \right). $$

We can also assume a Gaussian process prior on $x$ which, assuming a zero mean function, would take the form

$$ p(x) = \frac{1}{\sqrt{(2\pi)^n |K|}} \exp\left(-\frac{1}{2}x^\dagger K^{-1} x \right). $$

By Bayes' law, the posterior is given by the product of the likelihood and prior (up to a normalisation constant). We also know that this must have the form of a Gaussian so the term in the exponent of the posterior must, at least up to some constant $C$, be proportional to

$$ (x - \bar{x})^\dagger D^{-1} (x - \bar{x})  $$

where $\bar{x}$ is the posterior mean and $D$ is the posterior covariance. Expanding we find

$$ x^\dagger D^{-1} x - 2 x^\dagger D^{-1} \bar{x} + \bar{x}^\dagger D^{-1} \bar{x} + C  $$

where the factor of $2$ stems from the fact that $x^\dagger D^{-1} \bar{x} = \bar{x}^\dagger D^{-1} x$ since the term is a real scalar and we have included the constant $C$ for completeness. Also expanding the terms in the exponent of the (-log) likelihood times (-log) prior gives

$$ (y - x)^\dagger \Sigma^{-1} (y - x) + x^\dagger K^{-1} x = x^\dagger \left(\Sigma^{-1} + K^{-1}\right) x - 2 x^\dagger \Sigma^{-1} y + y^\dagger \Sigma^{-1} y $$

Now we can read off the unknowns by comparing coefficients. First, from the quadratic term, we have

$$ D^{-1} = \Sigma^{-1} + K^{-1}  $$

which is the sought after posterior covariance matrix. We can find its inverse in a numerically stable way using the matrix inversion lemma

$$ D = \Sigma - \Sigma (K + \Sigma)^{-1} \Sigma = \Sigma - \Sigma K^{-1}_y \Sigma, $$

where we have labelled $K_y = K + \Sigma$ for convenience.

Next, by comparing coefficients for the linear term, we find the posterior mean as

$$ D^{-1}\bar{x} = \Sigma^{-1} y ~~ \rightarrow ~~  \bar{x} = D \Sigma^{-1} y = (I - \Sigma K^{-1}_y) y = y - \Sigma K_y^{-1} y.   $$

So let's see how this works in practice. Let's first simulate some data.

In [ ]:
# choose a random domain
n = 100
t = 10*np.sort(np.random.rand(n))  # does not have to be sorted

# now we make up some complicated function that we want the Gaussian Process to model
x_true = 0.25*t**2 + 10*np.sin(t)

# add some noise
sigma_n = 1.0
noise = sigma_n* np.random.randn(n)
y = x_true + noise

# plot the noisy data
plt.plot(t, x_true, 'k')
plt.errorbar(t, y, yerr=sigma_n, fmt='xr', label='data')

In [ ]:
# we choose some arbitrary hyper-parameters to start with
sigmaf = 0.1
l = 1.0

# recompute the kernel on the new domain
K = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        K[i, j] = sqexp(t[i], t[j], sigmaf, l)

# construct the noise covariance matrix
Sigma = np.eye(n) * sigma_n**2
Sigma_inv = np.eye(n)/sigma_n**2  # because it is diagonal

# now we can compute the posterior mean and covariance
K_y = K + Sigma
# this is not always numerically stable (see use of SVD below)
xbar = y - Sigma.dot(np.linalg.solve(K_y, y))
D = Sigma - Sigma.dot(np.linalg.solve(K_y, Sigma))

# let's plot the posterior mean and covariance
plt.plot(t, x_true, 'k', label='true function')
plt.plot(t, xbar, 'b', label='posterior mean')
plt.fill_between(t, xbar - np.sqrt(np.diag(D)), xbar + np.sqrt(np.diag(D)), color='b', alpha=0.2, label='posterior std')
plt.legend()

# 1) - try altering the hyper-parameter values and see how the posterior changes
# 2) - try altering the noise level and see how the posterior changes
# 3) - draw samples from the posterior and plot them

# Hyper-parameter tuning

Clearly the posterior depends on the values of the hyper-parameters. These hyper-parameters must be tuned by maximising the marginal likelihood. The negative log of the marginal likelihood is given (up to some constants) by

$$ -\log p(y | \theta) = y^\dagger K_y^{-1} y + \log |K_y|, $$

note how we have encoded the implicit dependence on the hyper-parameters $\theta = [\sigma_f, l, \sigma_n]$ as a conditional distribution (strictly speaking we should also encode the dependence on the training points $t$ but we omit this for convenience). Training consists of minimising this function with respect to $\theta$. We'll do this with one of scipy's optimisation routines.   

In [ ]:
# since we'll be evaluating this function many times we provide a more efficient implementation
def sqexp2(r, sigmaf, l):  # here r = |t - tp|
    return sigmaf**2 * np.exp(-0.5 * r**2 / l**2)

tdiffs = np.abs(t[:, None] - t[None, :])

def marginal_likelihood(theta, y, tdiffs):
    n = t.size
    sigmaf = theta[0]
    l = theta[1]
    Ky = sqexp2(tdiffs, sigmaf, l) + np.eye(n)*theta[2]**2
    u, s, v = np.linalg.svd(Ky, hermitian=True)
    logdetK = np.sum(np.log(s))
    Kyinv = u.dot(v/s.reshape(n, 1))
    alpha = Kyinv.dot(y)
    # Z = (np.vdot(y, alpha) + logdetK)/2
    return np.vdot(y, alpha) + logdetK


# choose initial guess for hyper-parameters
theta0 = np.array([1.0, 1.0, 1.0])
res = minimize(marginal_likelihood, theta0, args=(y, tdiffs), bounds=[(1e-3, None), (1e-3, None), (1e-3, None)])
print(res)

In [ ]:
# we recompute the posterior mean and covariance using optimized hyper-parameters
theta = res.x
sigmaf = theta[0]
l = theta[1]
Sigma = np.eye(n) * theta[2]**2
Ky = sqexp2(tdiffs, sigmaf, l) + Sigma
u, s, v = np.linalg.svd(Ky, hermitian=True)
Kyinv = u.dot(v/s.reshape(n, 1))
xbar = y - Sigma.dot(Kyinv.dot(y))
D = Sigma - Sigma.dot(Kyinv.dot(Sigma))

# let's plot the posterior mean and covariance
plt.plot(t, x_true, 'k', label='true function')
plt.plot(t, xbar, 'b', label='posterior mean')
plt.fill_between(t, xbar - np.sqrt(np.diag(D)), xbar + np.sqrt(np.diag(D)), color='b', alpha=0.2, label='posterior std')
plt.legend()